# **The Molecular Treasure Hunt: Part 2: Docking a ligand library against each of your protein receptors**

## **This version uses the Vina external executable for the docking**

*An &#8491;ngstrom sized adventure by Sarah Harris (Leeds Physics) and Geoff Wells (UCL Pharmacy)*

# Running the Molecular Treasure Hunt!!

"*You cannot pass [your project]! I am a servant of the Secret Fire, wielder of the flame of Anor. The dark fire will not avail you, flame of Udûn [until you have submitted your report].*"

This procedure for understanding the amazing phenomenon of molecular recognition between biomolecules has been designed with a strong agenda. We believe that including the effect of biomolecular dynamics is key to improving our ability to make predictions about how biomolecules will interact. We can see these multiple conformational states either using NMR experiments, or assessing the diversity of X-ray crystal structures, or using theoretical tools such as atomistic molecular dynamics. You will therefore (we hope!!) include a number of protein structures in your "receptor" directory.

How drugs bind to multiple protein conformations depends upon the underlying physics and chemistry of proteins. In vivo, of course, this will also depend on the environment of the protein - e.g. whether it located in a large complex or in a membrane environment. For molecules to be orally bioavailable drugs, there are more stringent criteria (e.g. Lipinski's rule of 5). 

Thermal fluctuations (physics) drive conformational changes, but within the constraints imposed by the chemical structure of the proteins and the ligands. The requirement for the drug to reach the target site imposes additional criteria which depends on the physicochemical properties (e.g. clogP). Other considerations, such as the specificity of the compound for it's precise target, it's toxicity (related to specificity), synthetic accessibilty (e.g. can the chemists make it) and ease of formulation (e.g. can this be made into a safe and effective medicine, and is it feasible to make large quantities of it) are essential in practise, and can be non-trivial to overcome.   

Enjoy your Treasure Hunt!! Your search space encompasses the vast possibilities offered by the combinatorics of multiple protein conformations and enormous chemical diversity, particularly if you have embarked on a fragment library dock. Your limits imposed by pharmaceutical requirements though, are stringent, and in many ways unknown. Good luck!!   

We start by importing a number of python modules to run our calculations:

In [1]:
import sys
import os
import subprocess
#Importing glob is important for recursive filename searches - we use this to find all of our protein and ligand files!
from glob import glob
from pathlib import Path
from termcolor import colored
import multiprocessing as mp

import oddt

# Setting your ligand library and the protein receptors for your docking
Your working directory is the directory in which you opened the Jupyter notebook. Your proteins should be located in a folder called "receptor" within this directory. This notebook will dock every single ligand in your library against all of the protein structures (e.g. pdb files) that are in the "receptor" directory.

Before you start your treasure hunt, you must make sure that your proteins are **aligned** to each other, that they are **protonated** correctly and that you are happy with the position of the **grid box** that defines your receptor site for each protein structure. Check this by visualising in Chimera!!

### Protein files

In [3]:
#This defines our receptor structures (all of these files happen to have the same prefix, the starting structure (xal structure) is xxx_00.pdb)
receptor_filenames = glob('receptor/*.pdb')
#The sort command is *really* important for organising the output in the right order (the protein files are mixed up otherwise)
receptor_filenames.sort()
print(colored('Receptor filenames', 'blue', attrs=['bold']))
print(receptor_filenames)
print("Number of proteins is", len(receptor_filenames))

Receptor filenames
['receptor/5rlh_b.pdb', 'receptor/6zsl_prep.pdb', 'receptor/7rdx_prep.pdb', 'receptor/7rdy_prep.pdb']
Number of proteins is 4


This defines our ligand list as a global variable within the notebook...

### Ligand files

In [5]:
ligand_filenames = glob('/home/geoff/compounds/enamine_fragments/Minifrag/*.pdbqt')
ligand_filenames.sort()
print("Number of ligands is", len(ligand_filenames))

Number of ligands is 79


This tells us about how many processors we have available...

### Number of processors

In [2]:
num_cpu = mp.cpu_count()
print("Number of processors:", num_cpu)
#If necessary we can set the number of processors to be less than the mp.cpu_count()
#by commenting out the top line (by adding a # at the start of the line)
#and uncommenting the line below (by removing the # from the line) using a number you chose.
#only one of the two lines (EITHER the top or the bottom one) should be uncommented.
#num_cpu = 4

Number of processors: 12


Here we create a folder to store the results in - you can change the name of this, although it is best not to! (do not use spaces or puctuation other than underscores or hypens in your file name)!

### Results folder location

In [6]:
Results_Folder = 'MTH_Results'
rf = Path(Results_Folder)
try:
    rf.mkdir()
except FileExistsError as exc:
    print(colored('The results folder is already present - check that it is empty before starting!', 'blue',  attrs=['bold']))

## Making the protein files that we need for the docking calculations
### Preparing the protein .pdbqt files
The structures that we start with and align are .pdb files - these contain information on the atoms that are in the protein, the residue names and the x, y and z coordinates of each atom. For the docking calculations a modified version of the .pdb file is necessary that contains additional atom charge (q) and atom type (t) information - this is called a .pdbqt file. **Note**: the .pdbqt files do not contain non-polar hydrogens (this is a requirement for Autodock Vina-based docking calculations).

The next piece of code generates a .pdbqt file for each of your protein .pdb files. As long as your proteins don't contain very weird atoms or residues, this should work!! *This piece of code is complicated because we are subverting a process that Chimera usually hides from you! Chimera is running some of the code from MGLTools to execute these tasks, this is python2-based code, so for sustainability we have avoided the need to execute python2 code directly.*

In [4]:
#Let's use oddt to make the pdbqt files that we need...
#Load the aligned .pdb files into Chimera in sequence with the ligand and write out the corresponding .pdbqt files
print(colored('Making pdbqt files', 'blue', attrs=['bold']))
for receptor_filename in receptor_filenames:
    receptor_prefix = receptor_filename.split(".")[0]
    receptor_name = receptor_prefix.split("/")[1]
    receptor_pdbqt_file = receptor_prefix + '.pdbqt'
    print(receptor_name)
    rec = next(oddt.toolkit.readfile('pdb', receptor_filename))
    #oddt.docking.AutodockVina.write_vina_pdbqt(rec, receptor_pdbqt_file, flexible=False)
    rec.write('pdbqt', receptor_pdbqt_file, overwrite=True, opt={'r': None, 'c': None, 'h': None})
print(colored('Finished making pdbqt files', 'blue', attrs=['bold']))

Making pdbqt files
5rlh_b
6zsl_prep
7rdx_prep
7rdy_prep
Finished making pdbqt files


And here we get an estimate of how long the first step of the Molecular Treasure Hunt will take to run. As a very rough guess, you should get 1 dock every two minutes, so if your docking steps per cpu is 10, this should take around 20 minutes on a current (2020) workstation. Larger molecules are slower to dock (e.g the FDA library compared to the fragment library) because they have more rotatable bonds. 

In [7]:
docking_steps = len(ligand_filenames) * len(receptor_filenames)
print("Number of individual docking_steps is", docking_steps)
docking_steps_per_cpu = round((docking_steps / num_cpu), 2)
print("Number of docking steps per cpu is ", docking_steps_per_cpu)

Number of individual docking_steps is 316
Number of docking steps per cpu is  26.33


This defines a procedure for docking a series of ligands to a protein. It's horribly complicated - so only change this if you are feeling python-tastic!!

In [8]:
def dock_ligands(filename):
    #This line removes the filename suffix (.pdbqt)
    dock_input = filename.split(".")[0]
    #This line removes the filename prefix 
    dock_name = dock_input.split("/")[-1]
    #The next line runs a local docking experiment...
    #subprocess.run(["ls", "-l"])
    vina_receptor = PDBFile + '.pdbqt'
    vina_config = "receptor/conf.txt"
    vina_ligand = dock_input + '.pdbqt'
    vina_out = dock_name + '_out.pdbqt'
    vina_run = subprocess.run(["vina", "--receptor", vina_receptor, "--config", vina_config, "--ligand", vina_ligand, "--out", vina_out], stdout=subprocess.DEVNULL)
    
    #The input file for qvina "conf.txt" contains instructions to only output the top docked pose. However, it is 
    #also necessary to run the "qvina_split" command to produce a single ligand conformer, because of the flags that 
    #are present at the start and end of the pdbqt output file.
    
    vina_split = subprocess.run(["vina_split", "--input", vina_out], stdout=subprocess.DEVNULL)
    
    output_filenames = glob(dock_name + '_out*.pdbqt')
    #print(output_filenames)
    for output_filename in output_filenames:
        output_filename_prefix = output_filename.split(".")[0]
        output_filename_number_ligand = output_filename_prefix.split("out")[-1]
        output_filename_number = output_filename_prefix.split("_")[-1]
        if output_filename_number == 'out':
            output_filename_number = '00'
        else:
            with open(output_filename, 'r') as contents:
                for line in contents:
                    if line.startswith("REMARK VINA RESULT:"):
                        dock_energy = round(float(line.split()[3]),4)
            contents.close()
            print(dock_name + "_" + output_filename_number + " Autodock Vina Score =", dock_energy, " KCal/mol")
            top_pose = next(oddt.toolkit.readfile('pdbqt', output_filename))
            top_pose_out = top_pose.write('mol2')
            with open(PDBFile_name + "_" + dock_name + ".mol2",'a') as contents:
                contents.write("########## Name: " + dock_name + '_' + output_filename_number + '\n' + 
                         "########## Vina_BE: " + str(dock_energy) + '\n' + 
                        top_pose_out)
                contents.close()
    
    !'/bin/rm' '{dock_name}'_out*

Here we are performing the dock. We have the option to dock against multiple protein conformers, because we know that dynamics is important!!
You should not need to change this section, unless you want to do something special. 

In [9]:
#This runs a series of docking calculations on protein conformations found in the receptor folder
for receptor_filename in receptor_filenames:
    PDBFile = receptor_filename.split(".")[0]
    PDBFile_name = PDBFile.split("/")[1]
    print(colored('Docking for protein started', 'blue', attrs=['bold']))
    print(PDBFile_name)
   
    #Here we run the script that executes the docking procedure for our ligand list...
    #for ligand_filename in ligand_filenames:
    pool = mp.Pool(mp.cpu_count())
    pool.map(dock_ligands, [ligand_filename for ligand_filename in ligand_filenames])
    pool.close()
    print('Docking is complete for:',PDBFile_name)
    !'/bin/cat' '{PDBFile_name}'*.mol2 >> '{Results_Folder}'/VinaResults_'{PDBFile_name}'.mol2
    !'/bin/rm' '{PDBFile_name}'*.mol2

print(colored('All of the docking is complete!', 'blue', attrs=['bold']))


Docking for protein started
5rlh_b
Z104472692 conformer 02 Autodock Vina Score = -2.927  KCal/mol
Z104472692 conformer 08 Autodock Vina Score = -2.541  KCal/mol
Z104472692 conformer 09 Autodock Vina Score = -2.54  KCal/mol
Z104472692 conformer 03 Autodock Vina Score = -2.925 Z104507728 conformer 03 Autodock Vina Score = KCal/mol
 -3.06Z104472692 conformer 01 Autodock Vina Score =   KCal/mol-3.179 
 KCal/molZ104507728 conformer 10 Autodock Vina Score =
 Z104472692 conformer 04 Autodock Vina Score =-2.554  -2.871 KCal/mol
 Z104507728 conformer 04 Autodock Vina Score = KCal/mol 
-3.05Z104472692 conformer 10 Autodock Vina Score =  Z104472846 conformer 10 Autodock Vina Score =-2.533Z104494954 conformer 04 Autodock Vina Score =Z104474154 conformer 01 Autodock Vina Score =   KCal/mol  -2.88 KCal/mol-3.434
-2.883   Z104507728 conformer 08 Autodock Vina Score =
 KCal/mol KCal/molZ104472692 conformer 07 Autodock Vina Score =  KCal/mol
 -2.669

Z104474154 conformer 08 Autodock Vina Score = Z10449

Z1198152541 conformer 03 Autodock Vina Score = -3.747  KCal/mol
Z1198152541 conformer 05 Autodock Vina Score = -3.673  KCal/mol
Z1245635875 conformer 02 Autodock Vina Score = -4.671  KCal/mol
Z1245635875 conformer 01 Autodock Vina Score = -4.743  KCal/mol
Z1245635875 conformer 07 Autodock Vina Score = -4.151  KCal/mol
Z1245635875 conformer 08 Autodock Vina Score = -4.149  KCal/mol
Z1245635875 conformer 05 Autodock Vina Score = -4.24  KCal/mol
Z1245635875 conformer 03 Autodock Vina Score = -4.557  KCal/mol
Z1245635875 conformer 09 Autodock Vina Score = -4.068  KCal/mol
Z1245635875 conformer 04 Autodock Vina Score = -4.269Z1255486490 conformer 02 Autodock Vina Score =   KCal/mol-3.849
Z1245635875 conformer 10 Autodock Vina Score =  KCal/mol 
Z1255486490 conformer 10 Autodock Vina Score =-4.021  -3.506 KCal/mol
 Z1245635875 conformer 06 Autodock Vina Score = KCal/mol 
-4.176Z1255486490 conformer 03 Autodock Vina Score =   KCal/mol-3.841
  KCal/mol
Z1255486490 conformer 06 Autodock Vina Sc

Z415933552 conformer 03 Autodock Vina Score =Z2724335653 conformer 09 Autodock Vina Score =  -3.235-3.543   KCal/mol KCal/mol

Z2724335653 conformer 08 Autodock Vina Score =Z415933552 conformer 01 Autodock Vina Score =  -3.586 -3.397 KCal/mol 
 KCal/molZ33546369 conformer 01 Autodock Vina Score =Z2724335653 conformer 04 Autodock Vina Score =
  -3.43Z415933552 conformer 04 Autodock Vina Score = -3.789 -3.233  KCal/mol KCal/mol 
 KCal/molZ33546369 conformer 05 Autodock Vina Score =
Z2724335653 conformer 06 Autodock Vina Score =
  -3.623-3.177Z415933552 conformer 06 Autodock Vina Score =    KCal/mol-3.136 KCal/mol
 
 KCal/molZ2724335653 conformer 02 Autodock Vina Score =Z33546369 conformer 02 Autodock Vina Score =  
-3.883-3.353Z415933552 conformer 08 Autodock Vina Score =   KCal/mol  KCal/mol-3.082

Z2724335653 conformer 10 Autodock Vina Score = Z33546369 conformer 04 Autodock Vina Score =  KCal/mol -3.185
-3.516Z415933552 conformer 10 Autodock Vina Score =   KCal/mol  KCal/mol-3.02

 Z2

Z940713324 conformer 07 Autodock Vina Score = -3.082  KCal/mol
Z940713324 conformer 05 Autodock Vina Score = -3.183  KCal/mol
Z940713324 conformer 10 Autodock Vina Score = -2.953  KCal/mol
Z940713324 conformer 04 Autodock Vina Score = -3.196  KCal/mol
Z991531024 conformer 08 Autodock Vina Score = -3.535  KCal/mol
Z991531024 conformer 02 Autodock Vina Score = -3.895  KCal/mol
Z991531024 conformer 07 Autodock Vina Score = -3.536  KCal/mol
Z991531024 conformer 09 Autodock Vina Score = -3.518  KCal/mol
Z991531024 conformer 01 Autodock Vina Score = -4.019  KCal/mol
Z991531024 conformer 05 Autodock Vina Score = -3.661  KCal/mol
Z991531024 conformer 10 Autodock Vina Score = -3.495  KCal/mol
Z991531024 conformer 06 Autodock Vina Score = -3.542  KCal/mol
Z991531024 conformer 04 Autodock Vina Score = -3.688  KCal/mol
Z991531024 conformer 03 Autodock Vina Score = -3.878  KCal/mol
Docking is complete for: 5rlh_b
Docking for protein started
6zsl_prep
Z104472692 conformer 02 Autodock Vina Score = -2

Z1198152541 conformer 06 Autodock Vina Score =Z1238546206 conformer 09 Autodock Vina Score =  -4.206-3.311  KCal/mol 
 KCal/mol
Z1198152541 conformer 07 Autodock Vina Score = Z1238546206 conformer 02 Autodock Vina Score =-4.026  -3.686 KCal/mol 
 KCal/mol
Z1198152541 conformer 09 Autodock Vina Score =Z1238546206 conformer 08 Autodock Vina Score =  -3.915-3.416   KCal/mol KCal/mol

Z1198152541 conformer 08 Autodock Vina Score =Z1238546206 conformer 07 Autodock Vina Score =  -3.953-3.419   KCal/mol KCal/mol

Z1198152541 conformer 02 Autodock Vina Score =Z1238546206 conformer 01 Autodock Vina Score =  -3.687-4.354   KCal/mol KCal/mol

Z1238546206 conformer 04 Autodock Vina Score =Z1198152541 conformer 01 Autodock Vina Score =  -3.553-4.377   KCal/mol KCal/mol

Z1198152541 conformer 04 Autodock Vina Score =Z1238546206 conformer 05 Autodock Vina Score =  -3.541-4.271   KCal/mol KCal/mol

Z1198152541 conformer 03 Autodock Vina Score =Z1238546206 conformer 10 Autodock Vina Score =  -4.276-3.2

...

Z240126818 conformer 04 Autodock Vina Score = -3.341  KCal/mol
Z240126818 conformer 09 Autodock Vina Score =Z2517097658 conformer 08 Autodock Vina Score =  -3.041-3.022   KCal/mol
 KCal/mol
Z240126818 conformer 01 Autodock Vina Score =Z2517097658 conformer 06 Autodock Vina Score =  -3.439-3.078   KCal/mol KCal/mol

Z2517097658 conformer 01 Autodock Vina Score = -3.258  KCal/mol
Z2517097658 conformer 07 Autodock Vina Score = -3.054  KCal/mol
Z2517097658 conformer 09 Autodock Vina Score = -2.951  KCal/mol
Z2517097658 conformer 02 Autodock Vina Score = -3.201  KCal/mol
Z2517097658 conformer 10 Autodock Vina Score = -2.801  KCal/mol
Z2517097658 conformer 04 Autodock Vina Score = -3.095  KCal/mol
Z2517097658 conformer 03 Autodock Vina Score = -3.171  KCal/mol
Z2517097658 conformer 05 Autodock Vina Score =Z1459435569 conformer 05 Autodock Vina Score =  -3.079-3.315   KCal/mol KCal/mol

Z1459435569 conformer 04 Autodock Vina Score = -3.319  KCal/mol
Z1459435569 conformer 01 Autodock Vina Scor

 Z955123660 conformer 09 Autodock Vina Score =-3.354 -2.603   KCal/mol
 KCal/mol
Z955123660 conformer 02 Autodock Vina Score = -2.946  KCal/mol
Z992716898 conformer 10 Autodock Vina Score = -2.845  KCal/mol
Z992716898 conformer 09 Autodock Vina Score = -2.932  KCal/mol
Z992716898 conformer 07 Autodock Vina Score = -3.096  KCal/mol
Z992716898 conformer 06 Autodock Vina Score = -3.229  KCal/mol
Z992716898 conformer 01 Autodock Vina Score = -3.732  KCal/mol
Z992716898 conformer 02 Autodock Vina Score =Z803153778 conformer 10 Autodock Vina Score =  -3.532-3.338   KCal/mol KCal/mol

Z992716898 conformer 04 Autodock Vina Score = Z803153778 conformer 05 Autodock Vina Score =-3.394  -3.526  KCal/mol
Z803153778 conformer 07 Autodock Vina Score = -3.464  KCal/mol KCal/mol

Z992716898 conformer 05 Autodock Vina Score =Z803153778 conformer 01 Autodock Vina Score =  -3.391-3.877   KCal/mol KCal/mol

Z992716898 conformer 03 Autodock Vina Score =Z803153778 conformer 08 Autodock Vina Score =  -3.414-3

 Z104474140 conformer 02 Autodock Vina Score =Z1037576366 conformer 04 Autodock Vina Score =  KCal/mol
Z104472278 conformer 6 Autodock Vina Score =-3.038  -3.287 -3.429  KCal/mol  KCal/mol
 KCal/mol
Z104474140 conformer 10 Autodock Vina Score =
 Z104472278 conformer 1 Autodock Vina Score = -2.846-3.654  KCal/mol 
 KCal/mol
Z104474140 conformer 03 Autodock Vina Score = -2.938  KCal/mol
Z104474140 conformer 07 Autodock Vina Score = -2.873  KCal/mol
Z104474140 conformer 09 Autodock Vina Score = -2.849  KCal/mol
Z104474140 conformer 04 Autodock Vina Score = -2.936  KCal/mol
Z1184909877 conformer 04 Autodock Vina Score =Z104474140 conformer 01 Autodock Vina Score =  -3.429-3.043   KCal/mol KCal/mol

Z104474140 conformer 06 Autodock Vina Score = Z1184909877 conformer 09 Autodock Vina Score = -2.875-3.342   KCal/mol KCal/mol

Z1184909877 conformer 06 Autodock Vina Score =Z104474140 conformer 08 Autodock Vina Score =  -3.387-2.866   KCal/mol KCal/mol
Z1184909877 conformer 07 Autodock Vina Scor

Z1954800314 conformer 10 Autodock Vina Score = -2.733  KCal/mol
Z1954800314 conformer 01 Autodock Vina Score = -3.115  KCal/mol
Z1954800314 conformer 08 Autodock Vina Score = -2.755  KCal/mol
Z1954800314 conformer 07 Autodock Vina Score = -2.784  KCal/mol
Z1954800314 conformer 03 Autodock Vina Score = -3.035  KCal/mol
Z240126818 conformer 08 Autodock Vina Score = -3.114  KCal/mol
Z240126818 conformer 07 Autodock Vina Score = -3.176  KCal/mol
Z240126818 conformer 05 Autodock Vina Score = -3.233  KCal/mol
Z1459435569 conformer 05 Autodock Vina Score =Z240126818 conformer 03 Autodock Vina Score =  -3.376-3.309   KCal/mol KCal/mol

Z1459435569 conformer 04 Autodock Vina Score =Z240126818 conformer 10 Autodock Vina Score =  -3.395-3.054   KCal/mol
 KCal/mol
Z1459435569 conformer 01 Autodock Vina Score = Z240126818 conformer 06 Autodock Vina Score = -3.609 -3.218  KCal/mol
 KCal/molZ1459435569 conformer 09 Autodock Vina Score =
 Z240126818 conformer 02 Autodock Vina Score =-3.219 -3.417   KC

 -3.492Z59181971 conformer 03 Autodock Vina Score =   KCal/mol-3.666
 Z955123660 conformer 08 Autodock Vina Score = KCal/mol
 -2.598Z59181971 conformer 07 Autodock Vina Score =   KCal/mol-3.378 
 KCal/molZ955123660 conformer 03 Autodock Vina Score =
 Z59181971 conformer 09 Autodock Vina Score =-2.88 -3.343  KCal/mol 
 KCal/mol
Z955123660 conformer 01 Autodock Vina Score = -2.9  KCal/mol
Z955123660 conformer 06 Autodock Vina Score = -2.703  KCal/mol
Z955123660 conformer 07 Autodock Vina Score = -2.602  KCal/mol
Z955123660 conformer 10 Autodock Vina Score = -2.532  KCal/mol
Z955123660 conformer 04 Autodock Vina Score = -2.766  KCal/mol
Z955123660 conformer 05 Autodock Vina Score = -2.718  KCal/mol
Z955123660 conformer 09 Autodock Vina Score = -2.58  KCal/mol
Z955123660 conformer 02 Autodock Vina Score = -2.891  KCal/mol
Z992716898 conformer 10 Autodock Vina Score = -3.134  KCal/mol
Z992716898 conformer 09 Autodock Vina Score = -3.161  KCal/mol
Z992716898 conformer 07 Autodock Vina Score 

# Congratulations!!!
Your Treasure Hunt results are stored in the MTH_Results folder (unless you renamed it). You have a set of .mol2 files and .sdf files for each protein conformer. These have a VinaResults prefix. They contain the docked poses of all of your ligands in your library for each separate protein receptor configuration. 

"*None shall remember the deeds that are done in the last defence of your homes. Yet the deeds will not be less valiant because they are unpraised.*"

When you have these files - you are now ready for the next stage - have fun!

Sarah and Geoff

(an "out-of-office" $O^{3}P$ production)